In [1]:
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt
from scipy.sparse.linalg import norm as sparse_norm

%matplotlib inline

In [3]:
# generate relationship & special coefs matrix
m, n = 15000, 40000
relationship = np.random.rand(m, n)
relationship = np.where(relationship < 0.998, 0, 1)

In [5]:
special = np.random.rand(m, n)
special[relationship == 0] = 1 # if user didnt attach with item
special[special < 0.3] = 0
special[np.logical_and(0.3 <= special, special < 0.6)] = 5
special[np.logical_and(0.6 <= special, special < 1)] = 7

In [11]:
from scipy.sparse import csr_matrix

sparse_relationship = csr_matrix(relationship)

In [6]:
def als_functional(A, A_norm, U, VT):
    """
        Input
            A: sparse 2D array
            A_norm: Frobenius norm of A
            U, VT: 2D arrays such that U @ VT approximates A
        Output
            ||A - U VT||_F
    """
    # A.shape = (m, n); VT.shape = (r, n); U.shape = (m, r); 
    return np.sqrt(A_norm * A_norm + np.sum((U.T @ U) * (VT @ VT.T)) - 2 * np.sum((U.T @ A) * VT))


In [20]:
def iALS(A, C, rank, tolerance=1e-2):
    """
        Input
            A: 2D m x n numpy array
            C: 2D m x n numpy array: special coefs matrix 
            rank: required rank of the approximation
            tolerance: stop when delta_k is less or equal to it
            debug: print debug information on each iteration
            
        Output
            U, VT: m x rank, rank x n numpy arrays forming skeleton decomposition;
                   rows of matrix VT are orthonormal
            fs: list of f(U_k, VT_k)
            deltas: list of f(U_{k-1}, VT_{k-1}) - f(U_k, VT_{k})
            dists: list of distances from Im(U_{k-1}) and Im(U_k)
    """
    curr_delta, last_f = 1e2, 0
    A_upd = A.multiply(C) 
    A_norm = sparse_norm(A_upd)
    Q1_prev, Q_2 = np.zeros((A_upd.shape[0], rank)), np.eye(A_upd.shape[1], rank)
    while curr_delta > tolerance:
        Q_1, R_1 = np.linalg.qr(A_upd @ Q_2)    # Q_1.shape = (m, r); R_1.shape = (r, r)
        Q_2, R_2 = np.linalg.qr(A_upd.T @ Q_1)  # Q_2.shape = (n, r); R_2.shape = (r, r)
        curr_f = als_functional(A_upd, A_norm, Q_1 @ R_2.T, Q_2.T)
        curr_delta = curr_f - last_f
        last_f = curr_f
        Q1_prev = Q_1
    return Q_1, (R_2.T @ Q_2.T) # v_{ui} * c_{ui}


In [21]:
user_matrix, item_matrix = iALS(sparse_relationship, special, 30)
# sparse_relationship.shape, special.shape
# sparse_relationship.multiply(special)

In [42]:
def recommend(score_vec, pred_am, V):
    """
        Input
            score_vec: (n_movies, ) numpy array // do we need batch_size users recs?
            pred_am: requested number of recommendations
            V: 2D numpy array (n_movies x rank)
            
        Output
            recs: batch_size x pred_am array of movies to recommend, with descending predicted rating
    """
    Q, R = np.linalg.qr(V)
    mat_recs = ((np.array([score_vec]) @ Q) @ (R @ np.linalg.inv(R.T @ R) @ R.T)) @ Q.T
    recs = np.argsort(-mat_recs, axis=1)[:, :pred_am]
    return recs[0]


In [32]:
users_batch_vec = np.random.rand(n)
users_batch_vec = np.where(users_batch_vec < 0.998, 0, 1)

In [46]:
np.argsort(-users_batch_vec, axis=0)

array([39026, 24490, 17629, ..., 13349, 13342, 39999], dtype=int64)

In [40]:
np.array([users_batch_vec]).shape, item_matrix.shape

((1, 40000), (30, 40000))

In [48]:
recs_count = 10
user_recs = recommend(users_batch_vec, recs_count, item_matrix.T)

In [44]:
print(user_recs)

[ 7 16 14  2 13 22 26 25 23 21]
